In [1]:
from pydantic import BaseModel
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
import ollama
from langchain.llms.base import LLM

e:\Finalyear_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# load the pdf



In [2]:
loader = PyPDFLoader("example1.pdf")
documents = loader.load()


# Embeddings 

In [3]:

token_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=50) 
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
texts = token_splitter.split_documents(documents)

C:\Users\rajpa\AppData\Local\Temp\ipykernel_9884\298322004.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')


In [4]:
db = FAISS.from_documents(texts,embedding = embedding_model)

In [5]:
# Define a custom LLM wrapper for Ollama to integrate with LangChain
class OllamaLLM(LLM, BaseModel):
    model_name: str  # Explicitly declare the model_name field

    def _call(self, prompt: str, stop=None):
        response = ollama.chat(model=self.model_name, messages=[{"role": "user", "content": prompt}])
        print(response)
        return response["response"]

    @property
    def _llm_type(self):
        return "ollama"

In [6]:

# Define the prompt template
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Initialize the Ollama model using the custom LLM wrapper
ollama_llm = OllamaLLM(model_name="llama3.2")

# Define the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=ollama_llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

# Define the RAG pipeline function
def rag_pipeline(query):
    result = qa({"query": query})
    answer = result['result']
    source_documents = result['source_documents']
    context = [doc.page_content for doc in source_documents]
    return  answer

In [7]:
print(rag_pipeline("Write a R program to find the sum of all even numbers between two given numbers using for loop."))

C:\Users\rajpa\AppData\Local\Temp\ipykernel_9884\299388212.py:27: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


model='llama3.2' created_at='2025-02-21T05:39:07.7310562Z' done=True done_reason='stop' total_duration=232428485400 load_duration=7449937400 prompt_eval_count=1175 prompt_eval_duration=161505000000 eval_count=261 eval_duration=63363000000 message=Message(role='assistant', content='Here is a simple R program that uses a `for` loop to calculate the sum of all even numbers between two given numbers:\n\n```r\n# Function to find sum of even numbers\nfind_sum_even <- function(start, end) {\n  sum_even <- 0\n  for (i in start:end) {\n    if (i %% 2 == 0) { # Check if number is even\n      sum_even <- sum_even + i\n    }\n  }\n  return(sum_even)\n}\n\n# Example usage:\nstart_num <- 1\nend_num <- 20\n\nsum_result <- find_sum_even(start_num, end_num)\n\nprint(sum_result) # Output: The sum of all even numbers between start_num and end_num.\n```\n\nThis program defines a function `find_sum_even` that takes two numbers as arguments. It initializes the sum variable to zero, then uses a `for` loop to

KeyError: 'response'